# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import folium
import datetime
import pandas as pd
from folium.plugins import HeatMap

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
# df = cda.crash_dataframe(verbose=True)

scooter rider injured in hit and run 

https://www.denver7.com/news/local-news/denver-woman-recovering-from-serious-injuries-after-hit-and-run-while-riding-scooter

incident_id = 2023182255

In [3]:
# University Blvd and E 3rd Ave (2022,12,14) incident_id = 2022639830
# lat_lon = [39.720934, -104.959436]

# Speer Blvd and W 11th Ave
# lat_lon = [39.73378086237446, -104.99382870794332]

# Sheridan and W 38th Ave
# lat_lon = [39.76932995840248, -105.05322879106377]

# 18th St and Wazee - 2023-01-07
# lat_lon = [39.753095972145644, -104.99765515593928]

# 17th Ave and Humboldt
# lat_lon = [39.74325582993406, -104.96969463423684]

# Evans and Holly
# lat_lon = [39.678456222120396, -104.92219658639499]

# Perry and 32nd
# lat_lon = [39.76207714280117, -105.03931855940714]

# W 23rd Ave and Julian
# lat_lon = [39.751187102730654, -105.03155988650708]

# 35th St and Welton
# lat_lon = [39.76762137425839, -104.97535848218475]

# 1776 Race St
lat_lon = [39.744484814811436, -104.96299454652576]

# 730 S Canosa Ct
# lat_lon = [39.70332702509137, -105.01894998129806]

# Sherman St house
# lat_lon = [39.721383, -104.984736]

# Broadway and Colfax - scooter rider injured in hit and run
# lat_lon = [39.74012825562107, -104.98743057571703]

# lat_lon = cda.geocode_location('W 14th Ave & Federal Blvd, Denver, CO')
# lat_lon = cda.geocode_location('W 32nd Ave & Speer Blvd, Denver, CO')
lat_lon = cda.geocode_location('Columbine and E 14th Ave, Denver, CO')

In [4]:
# crash occurred within this distance of the given latitude and longitude
radius_miles = 0.05

query = f"""
select
*
, reported_date at time zone 'America/Denver' as reported_date_local
, ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
) as distance_feet

from crashes

where geo_lon is not null
and ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
    ) < {radius_miles * 5280}

order by reported_date
"""
# and extract(year from reported_date) >= 2022

nearby = pd.read_sql(query, cda.conn)

In [5]:
nearby.groupby(nearby.reported_date.dt.year).agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date,,,,,
2013,6,0,0,0,0
2014,5,0,0,0,0
2015,6,0,0,0,0
2016,7,0,0,0,0
2017,5,0,0,0,0
2018,6,0,0,0,0
2019,3,0,0,0,0
2020,1,0,0,0,0
2021,4,0,0,0,0


In [6]:
nearby['all_time'] = 'all time'
nearby.groupby('all_time').agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
all_time,,,,,
all time,55,0,0,0,0


In [7]:
display_cols = [
    'reported_date_local'
    , 'top_traffic_accident_offense'
    , 'incident_address_corrected'
    , 'bicycle_ind'
    , 'pedestrian_ind'
    , 'sbi'
    , 'fatality'
    , 'distance_feet'
    , 'incident_id'
]
# nearby.to_clipboard()
# nearby[display_cols]

In [8]:
nearby[display_cols] #.tail(5)

,reported_date_local,top_traffic_accident_offense,incident_address_corrected,bicycle_ind,pedestrian_ind,sbi,fatality,distance_feet,incident_id
0,2013-04-02 14:00:00.000000,TRAF - ACCIDENT,2500 BLOCK E 14TH AVE,0.0,0.0,False,False,165.197802,2013147504
1,2013-04-13 15:00:00.000000,TRAF - ACCIDENT - HIT & RUN,1400 BLOCK N COLUMBINE ST,0.0,0.0,False,False,110.955259,2013173782
2,2013-06-02 11:43:00.000000,TRAF - ACCIDENT - HIT & RUN,2400 BLOCK E 14TH AVE,0.0,0.0,False,False,178.349992,2013247889
3,2013-06-06 16:41:00.000001,TRAF - ACCIDENT,2500 BLOCK E 14TH AVE,0.0,0.0,False,False,165.197802,2013254880
4,2013-09-12 17:22:00.000000,TRAF - ACCIDENT,E 14TH AVE / N COLUMBINE ST,0.0,0.0,False,False,0.758492,2013437773
5,2013-09-28 16:54:00.000001,TRAF - ACCIDENT,2400 BLOCK E 14TH AVE,0.0,0.0,False,False,178.349992,2013467595
6,2014-01-16 13:59:00.000000,TRAF - ACCIDENT - HIT & RUN,1400 N COLUMBINE ST,0.0,0.0,False,False,110.955259,201426694
7,2014-02-03 19:47:00.000000,TRAF - ACCIDENT,E 14TH AVE / N COLUMBINE ST,0.0,0.0,False,False,0.758492,201458736
8,2014-07-11 17:03:00.000000,TRAF - ACCIDENT - HIT & RUN,1355 N COLUMBINE ST,0.0,0.0,False,False,263.851977,2014358246
9,2014-12-15 16:17:00.000000,TRAF - ACCIDENT - HIT & RUN,E 14TH AVE / N COLUMBINE ST,0.0,0.0,False,False,0.758492,2014663933


In [9]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(nearby[['geo_lat', 'geo_lon']]).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map

In [10]:
cda.conn.dispose()